In [ ]:
// bfs
#include<iostream>
#include<unordered_map>
#include<vector>
#include<queue>
#include<omp.h>
#include<chrono>

using namespace std;

std::unordered_map<int, vector<int>> graph;
std::vector<bool> visited;

void serial_bfs(int node) {
    queue<int> temp_queue;
    temp_queue.push(node);
    
    while(!temp_queue.empty()) {
        int current = temp_queue.front();
        temp_queue.pop();

        if(!visited[current]) {
            visited[current] = true;
            cout<<current<<" ";

            for (int adjacent : graph[current]) {
                if (!visited[adjacent]) {
                    temp_queue.push(adjacent);
                }
            }
        }
    }
}

void parallel_bfs(int node) {
    queue<int> temp_queue;
    temp_queue.push(node);

    while(!temp_queue.empty()) {
        int current = temp_queue.front();
        temp_queue.pop();

        if(!visited[current]) {
            visited[current] = true;
            cout<<current<<" ";

            #pragma omp parallel for
            for (int adjacent : graph[current]) {
                if (!visited[adjacent]) {
                    #pragma omp critical
                    {
                      temp_queue.push(adjacent);
                    }
                }
            }
        }
    }
}

int main() {
    int edges, nodes, initial, u, v;

    cout<<"Enter No of Nodes: ";
    cin>>nodes;
    visited.assign(nodes, false);

    cout<<"Enter No of Edges: ";
    cin>>edges;

    cout<<"Enter Pair of Edges:"<<endl;
    for(int i=0; i<edges; i++) {
        cin>>u>>v;
        graph[u].push_back(v);
    }

    cout<<"Enter Start Node: ";
    cin>>initial;

    auto start_s = chrono::high_resolution_clock::now();
    cout<<"Serial BFS: ";
    serial_bfs(initial);
    auto end_s = chrono::high_resolution_clock::now();
    chrono::duration<double, milli> time_s = end_s - start_s;
    cout<<"\nTime: "<<time_s.count()<<"ms"<<endl;
    
    visited.assign(nodes, false);

    auto start_p = chrono::high_resolution_clock::now();
    cout<<"Parallel BFS: ";
    parallel_bfs(initial);
    auto end_p = chrono::high_resolution_clock::now();
    chrono::duration<double, milli> time_p = end_p - start_p;
    cout<<"\nTime: "<<time_p.count()<<"ms"<<endl;

    return 0;
}

// dfs

#include<iostream>
#include<unordered_map>
#include<vector>
#include<omp.h>
#include<chrono>

using namespace std;

std::unordered_map<int, vector<int>> graph;
std::vector<bool> visited;

void serial_dfs(int node) {
    if(!visited[node]) {
        visited[node] = true;
        cout<<node<<" ";
        
        for (int adjacent : graph[node]) {
            if (!visited[adjacent]) {
                serial_dfs(adjacent);
            }
        }
    }
}

void parallel_dfs(int node) {
    if(!visited[node]) {
        visited[node] = true;
        cout<<node<<" ";
        
        #pragma omp parallel for
        for (int adjacent : graph[node]) {
            if (!visited[adjacent]) {
              #pragma omp task
              {
                parallel_dfs(adjacent);
              }
            }
        }
    }
}

int main() {
    int edges, nodes, initial, u, v;

    cout<<"Enter No of Nodes: ";
    cin>>nodes;
    visited.assign(nodes, false);

    cout<<"Enter No of Edges: ";
    cin>>edges;

    cout<<"Enter Pair of Edges:"<<endl;
    for(int i=0; i<edges; i++) { 
        cin>>u>>v;
        graph[u].push_back(v);
    }

    cout<<"Enter Start Node: ";
    cin>>initial;

    auto start_s = chrono::high_resolution_clock::now();
    cout<<"Serial DFS: ";
    serial_dfs(initial);
    auto end_s = chrono::high_resolution_clock::now();
    chrono::duration<double, milli> time_s = end_s - start_s;
    cout<<"\nTime: "<<time_s.count()<<"ms"<<endl;
    
    visited.assign(nodes, false);

    auto start_p = chrono::high_resolution_clock::now();
    cout<<"Parallel DFS: ";
    parallel_dfs(initial);
    auto end_p = chrono::high_resolution_clock::now();
    chrono::duration<double, milli> time_p = end_p - start_p;
    cout<<"\nTime: "<<time_p.count()<<"ms"<<endl;

    return 0;
}

// bubble

#include<iostream>
#include<omp.h>
#include<chrono>

using namespace std;

void swap(int &a, int &b) {
    int temp = a;
    a = b;
    b = temp;
}

void serial_bubble(int *arr, int n) {
    bool swapped;
    for (int i=0; i<n-1; i++) {
        swapped = false;
        for (int j=0; j<n-i-1; j++) {
            if (arr[j] > arr[j+1]) {
                swap(arr[j], arr[j+1]);
                swapped = true;
            }
        }
        if(!swapped) {
            break;
        }
    }
}

void parallel_bubble(int *arr, int n) {
    bool swapped;
    // #pragma omp parallel for
    for (int i=0; i<n-1; i++) {
        swapped = false;

        #pragma omp parallel for shared(arr, swapped)
        for (int j=0; j<n-i-1; j++) {
          if (arr[j] > arr[j+1]) {
            #pragma omp critical
            {
              swap(arr[j], arr[j+1]);
              swapped = true;
            }
          }
        }
            
        if(!swapped) {
            break;
        }
    }
}

void print_list(int *arr, int n) {
    for(int i=0; i<n; i++) {
        cout<<arr[i]<<" ";
    }
}

int main() {
    int *list1, *list2, e, n;
    cout<<"Enter No of Elements: ";
    cin>>n;
    list1 = list2 = new int[n];

    cout<<"Enter Elements: "<<endl;
    for(int i=0; i<n; i++) {
   	    cin>>e;
        list1[i] = list2[i] = e;
    }

    auto start_s = chrono::high_resolution_clock::now();
    serial_bubble(list1, n);
    auto end_s = chrono::high_resolution_clock::now();
    chrono::duration<double, milli> time_s = end_s - start_s;
    
    auto start_p = chrono::high_resolution_clock::now();
    parallel_bubble(list2, n);
    auto end_p = chrono::high_resolution_clock::now();
    chrono::duration<double, milli> time_p = end_p - start_p;

    cout<<"Serial Bubble Sort: ";
    print_list(list1, n);
    cout<<"\nTime: "<<time_s.count()<<"ms"<<endl;
    cout<<"Parallel Bubble Sort: ";
    print_list(list2, n);
    cout<<"\nTime: "<<time_p.count()<<"ms"<<endl;

    return 0;
}

// merge

#include<iostream>
#include<omp.h>
#include<chrono>

using namespace std;

void merge(int list[], int first, int mid, int last) {
	int temp[last+1];
	int i = first, j = mid+1, k = 0;

	// #pragma omp parallel for
	while(i<=mid || j<=last) {
		if(i<=mid && j<=last) {
			if(list[i] < list[j]) {
				temp[k++] = list[i++];
			}
			else {
				temp[k++] = list[j++];
			}
		}
		else if(i<=mid) {
			temp[k++] = list[i++];
		}
		else if(j<=last) {
			temp[k++] = list[j++];
		}
	}

	for(i=first, j=0; i<=last; i++, j++) {
		list[i] = temp[j];
	}
}

void mergesort(int list[], int first, int last) {
	if(first < last) {
    	int mid = (first + last) / 2;
    	#pragma omp parallel sections
    	{
        	#pragma omp section
        	{
            	mergesort(list, first, mid);
        	}
        	#pragma omp section
        	{
            	mergesort(list, mid+1, last);
        	}
    	}
    	merge(list, first, mid, last);
	}
}

void printList(int list[], int n) {
	for(int i=0; i<n; i++) {
		cout<<list[i]<<" ";
	}
}


int main() {
	int *list, n;
	cout<<"Enter No of Elements: ";
	cin>>n;
	list = new int[n];

	cout<<"Enter Elements:"<<endl;
	for(int i=0; i<n; i++) {
    	cin>>list[i];
	}
  auto start_p = chrono::high_resolution_clock::now();
	mergesort(list, 0, n-1);
  auto end_p = chrono::high_resolution_clock::now();
  chrono::duration<double, milli> time_p = end_p - start_p;
	
	cout<<"Sorted Array: ";
	printList(list, n);
  cout<<"\nExecution Time: "<<time_p.count()<<"ms"<<endl;

	return 0;
}

// min-max

#include <iostream>
#include <omp.h>
#include <climits>

using namespace std;

void min_reduction(int list[], int n) {
  int min = INT_MAX;

  #pragma omp parallel for reduction(min: min)
  for (int i = 0; i < n; i++) {
    if (list[i] < min) {
      min = list[i];
    }
  }

  cout<<"Minimum Value: "<<min<<endl;
}

void max_reduction(int list[], int n) {
  int max = INT_MIN;

  #pragma omp parallel for reduction(max: max)
  for (int i=0; i<n; i++) {
    if (list[i] > max) {
      max = list[i];
    }
  }

  cout<<"Maximum value: "<<max<<endl;
}

void sum_reduction(int list[], int n) {
  int sum = 0;

  #pragma omp parallel for reduction(+: sum)
  for (int i=0; i<n; i++) {
	  sum += list[i];
  }

  cout<<"Sum: "<<sum<<endl;
}

void avg_reduction(int list[], int n) {
  int sum = 0;

  #pragma omp parallel for reduction(+: sum)
  for (int i=0; i<n; i++) {
	  sum += list[i];
  }

  cout<<"Average: "<<(double)sum/(n-1)<<endl;
}

int main() {
  int *list, n;

  cout<<"Enter No of Elements: ";
  cin>>n;
  list = new int[n];

  cout<<"Enter Elements:"<<endl;
  for(int i=0; i<n; i++) {
    cin>>list[i];
  }

  min_reduction(list, n);
  max_reduction(list, n);
  sum_reduction(list, n);
  avg_reduction(list, n);
}

// CUDA multi

/* !pip install git+https://github.com/afnan47/cuda.git */
// %load_ext nvcc_plugin

%%cu
#include <iostream>
using namespace std;


// CUDA code to multiply matrices
__global__ void multiply(int* A, int* B, int* C, int size) {
    // Uses thread indices and block indices to compute each element
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < size && col < size) {
        int sum = 0;
        for (int i = 0; i < size; i++) {
            sum += A[row * size + i] * B[i * size + col];
        }
        C[row * size + col] = sum;
    }
}


void initialize(int* matrix, int size) {
    for (int i = 0; i < size * size; i++) {
        matrix[i] = rand() % 10;
    }
}


void print(int* matrix, int size) {
    for (int row = 0; row < size; row++) {
        for (int col = 0; col < size; col++) {
            cout << matrix[row * size + col] << " ";
        }
        cout << '\n';
    }
    cout << '\n';
}


int main() {
    int* A, * B, * C;

    int N = 2;
    int blockSize =  16;

    int matrixSize = N * N;
    size_t matrixBytes = matrixSize * sizeof(int);

    A = new int[matrixSize];
    B = new int[matrixSize];
    C = new int[matrixSize];

    initialize(A, N);
    initialize(B, N);
    cout << "Matrix A: \n";
    print(A, N);

    cout << "Matrix B: \n";
    print(B, N);

    
    int* X, * Y, * Z;
    // Allocate space
    cudaMalloc(&X, matrixBytes);
    cudaMalloc(&Y, matrixBytes);
    cudaMalloc(&Z, matrixBytes);

    // Copy values from A to X
    cudaMemcpy(X, A, matrixBytes, cudaMemcpyHostToDevice);
    
    // Copy values from A to X and B to Y
    cudaMemcpy(Y, B, matrixBytes, cudaMemcpyHostToDevice);

    // Threads per CTA dimension
    int THREADS = 2;

    // Blocks per grid dimension (assumes THREADS divides N evenly)
    int BLOCKS = N / THREADS;

    // Use dim3 structs for block  and grid dimensions
    dim3 threads(THREADS, THREADS);
    dim3 blocks(BLOCKS, BLOCKS);

    // Launch kernel
    multiply<<<blocks, threads>>>(X, Y, Z, N);

    cudaMemcpy(C, Z, matrixBytes, cudaMemcpyDeviceToHost);
    cout << "Multiplication of matrix A and B: \n";
    print(C, N);

    delete[] A;
    delete[] B;
    delete[] C;

    cudaFree(X);
    cudaFree(Y);
    cudaFree(Z);

    return 0;
}

// CUDA addition

/* !pip install git+https://github.com/afnan47/cuda.git */
// %load_ext nvcc_plugin

%%cu
#include <iostream>
using namespace std;

__global__ void add(int* A, int* B, int* C, int size) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;

    if (tid < size) {
        C[tid] = A[tid] + B[tid];
    }
}


void initialize(int* vector, int size) {
    for (int i = 0; i < size; i++) {
        vector[i] = rand() % 10;
    }
}

void print(int* vector, int size) {
    for (int i = 0; i < size; i++) {
        cout << vector[i] << " ";
    }
    cout << endl;
}

int main() {
    int N = 4;
    int* A, * B, * C;

    int vectorSize = N;
    size_t vectorBytes = vectorSize * sizeof(int);

    A = new int[vectorSize];
    B = new int[vectorSize];
    C = new int[vectorSize];

    initialize(A, vectorSize);
    initialize(B, vectorSize);

    cout << "Vector A: ";
    print(A, N);
    cout << "Vector B: ";
    print(B, N);

    int* X, * Y, * Z;
    cudaMalloc(&X, vectorBytes);
    cudaMalloc(&Y, vectorBytes);
    cudaMalloc(&Z, vectorBytes);

    cudaMemcpy(X, A, vectorBytes, cudaMemcpyHostToDevice);
    cudaMemcpy(Y, B, vectorBytes, cudaMemcpyHostToDevice);

    int threadsPerBlock = 256;
    int blocksPerGrid = (N + threadsPerBlock - 1) / threadsPerBlock;

    add<<<blocksPerGrid, threadsPerBlock>>>(X, Y, Z, N);

    cudaMemcpy(C, Z, vectorBytes, cudaMemcpyDeviceToHost);

    cout << "Addition: ";
    print(C, N);

    delete[] A;
    delete[] B;
    delete[] C;

    cudaFree(X);
    cudaFree(Y);
    cudaFree(Z);

    return 0;
}